# 14.TRPO （Trust Region Policy Optimization）：信任区域策略优化
> 即使 **Actor-Critic** 在一定程度上能够降低训练过程中的方差，在实际应用过程中仍与传统策略梯度算法一样存在着 **训练不稳定** 的情况。根本原因在于策略更新过大：网络参数对样本传递的环境信息 **“过度拟合”**，导致训练时全局策略变差，造成训练的不稳定。特别是当面临情况复杂，策略网络是深度模型时，沿着策略梯度更新参数，步长太长，在一段时间内陷入对部分样本的过分信任，会导致全局策略突然显著变差。

> **TRPO（Trust Region Policy Optimization）算法** 最初由 John Schulman 等人于2015年在 [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477) 中提出。
> 论文引入了 **信任区域（trust region）和KL散度约束** 的概念。在 **信任区域** 上更新策略时能够得到某种策略性能的安全性保证，是其主要思想。**TRPO** 描述了一种优化策略的迭代过程：在理论上能够保证策略学习的 **性能单调性**，并在实际应用中取得了比策略梯度算法更好的效果。

## 14.1 TRPO原理
![TRPO原理示意图](Illustrations/TRPO原理示意图.png)
- 左图表示当**完全不设置信任区域**时，传统策略的梯度更新可能导致策略的性能骤降
- 右图表示当**设置了信任区域** 时，可以保证每次策略的梯度更新都能来带性能的提升

### 14.1.1 目标函数
> **TRPO** 对目标函数进行了变化：

> 无论策略为什么，采样一条轨迹的**初始状态$s_0$的分布$p(s_0)$固定和策略$\pi$无关**（实际实验中也规定如此），也就是说:
$$\begin{aligned}
J(\theta) & =\mathbb{E}_{\pi}[V^{\pi_\theta}(s_0)] =\sum_{s_0}p(s_0)V^{\pi_\theta}(s_0) \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}[V^{\pi_\theta}(s_0)] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tV^{\pi_\theta}(s_t)-\sum_{t=1}^\infty\gamma^tV^{\pi_\theta}(s_t)\right] \\
 & =-\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^{\infty}\gamma^{t}\left(\gamma V^{\pi_{\theta}}(s_{t+1})-V^{\pi_{\theta}}(s_{t})\right)\right]
\end{aligned}$$

> 基于以上等式，可以推导新旧策略的目标函数之间的差距：
$$\begin{aligned}
J(\theta^{\prime})-J(\theta) & =\mathbb{E}_{\pi}\left[V^{\pi_{\theta^{\prime}}}(s_0)\right]-\mathbb{E}_{\pi}\left[V^{\pi_\theta}(s_0)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tr(s_t,a_t)\right]+\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left(\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right)\right] \\
 & =\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^t\left[r(s_t,a_t)+\gamma V^{\pi_\theta}(s_{t+1})-V^{\pi_\theta}(s_t)\right]\right]
\end{aligned}$$


> 可见上述公式存在 **时序差分残差** 形式，以下转换成**优势函数A**：
$$J(\theta^{\prime})-J(\theta)=\mathbb{E}_{\pi_{\theta^{\prime}}}\left[\sum_{t=0}^\infty\gamma^tA^{\pi_\theta}(s_t,a_t)\right]$$

> 展开期望形式：
$$J(\theta^{\prime})-J(\theta)=\sum_\tau\left[p(\tau|\pi_{\theta^{\prime}})\sum_{t=0}^\infty\gamma^tA^{\pi_\theta}(s_t,a_t)\right]$$
- 轨迹概率：$p(\tau|\theta)=p(s_0)\prod_{t=0}^{T}[\pi_\theta(a_t|s_t)p(s_{t+1}|s_t,a_t)]$ 
> 又状态访问分布:$\nu^\pi(s)=(1-\gamma)\sum_{t=0}^\infty\gamma^tP(s_t = s, a_t = a \| \pi)$，转换为状态访问分布表达形式：
$$
J(\theta^{\prime})-J(\theta) = \frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta'}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right]
$$

> 所以只要保证:
$$\sum_{s} \left[ \nu^{\pi_{\theta'}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right] \geq 0$$
> 就能保证策略性能的单调递增。

*但是把所有可能的**新策略**都拿来采样数据去**获取状态访问分布**，然后判断哪个**新策略**满足上述条件的做法显然是**不现实**的*
> **TRPO** 做了一步近似操作，**忽略**两个策略之间的状态访问分布变化，直接**采用旧的策略的状态分布$\nu^{\pi_{\theta}}(s)$**：
$$J(\theta^{\prime})-J(\theta) = \frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right]$$

- 当新旧策略非常接近时，状态访问分布变化很小，这么近似是合理的
> 如此再定义**优化目标**:
$$\begin{aligned}
L_\theta(\theta^{\prime})& =J(\theta)+\frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a} \left[ \pi_{\theta'}(a | s)   A^{\pi_\theta}(s, a)  \right] \right] \\
& =J(\theta)+\frac{1}{1 - \gamma} \sum_{s} \left[ \nu^{\pi_{\theta}}(s) \sum_{a}\pi_{\theta}(a | s) \left[    \frac{\pi_{\theta^{\prime}}(a|s)}{\pi_\theta(a|s)}A^{\pi_\theta}(s,a) \right] \right] \\
& =J(\theta)+\mathbb{E}_{s\sim\nu^{\pi_\theta}}\mathbb{E}_{a\sim\pi_\theta(\cdot|s)}\left[\frac{\pi_{\theta^{\prime}}(a|s)}{\pi_\theta(a|s)}A^{\pi_\theta}(s,a)\right]
\end{aligned}$$

### 14.1.2 约束

#### Kullback-Leibler (KL) 散度：用于衡量一个概率分布相对于另一个分布的信息丧失
> 给定两个概率分布 $\( P \)$ 和 $\( Q \)$，其中 $\( P \)$ 通常代表真实分布（或“目标分布”），$\( Q \)$ 代表近似分布（或“预测分布”）
**KL 散度**衡量了从分布 $\( Q \)$ 生成数据时，丢失多少来自真实分布 $\( P \)$ 的信息。KL 散度越小，表示两个分布越相似；如果 $\( P \)$ 和 $\( Q \)$ 完全相同，则 KL 散度为零。

> 对于离散型概率分布：
$$
\[
\text{KL}(P || Q) = \sum_{x} P(x) \log \left( \frac{P(x)}{Q(x)} \right)
\]
$$
> 对于连续型概率分布：
$$
\[
\text{KL}(P || Q) = \int_{-\infty}^{\infty} p(x) \log \left( \frac{p(x)}{q(x)} \right) dx
\]
$$
- $\( P(x) \)$ 是真实分布的概率密度函数；
- $\( Q(x) \)$ 是近似分布的概率密度函数；

##### 主要性质：
1. 非负性：KL 散度总是非负的：只有当 $\( P = Q \)$ 时，KL 散度才为零。
   $$\[
   \text{KL}(P || Q) \geq 0
   \]$$
2. 非对称性：KL 散度是非对称的：这意味着 KL 散度不能当作一个真正的“距离”度量。
   $$\[
   \text{KL}(P || Q) \neq \text{KL}(Q || P)
   \]$$
3. 可加性：如果将一个复杂问题分解为多个子问题，KL 散度是可加的。

---
在 **TRPO** 中，**KL 散度** 用于限制每次策略更新的幅度，确保新的策略与旧策略之间的差异不会太大，从而保持优化过程的稳定性。具体来说，**TRPO** 在每次更新策略参数时加入一个基于 **KL 散度** 的 **约束** ：
$$
\[
\sum_s\nu^{\pi_\theta}(s)\mathrm{KL}\left[\pi_\theta(\cdot|s)||\pi_{\theta^{\prime}}(\cdot|s)\right]=\mathbb{E}_{s\sim\nu^{\pi_\theta}} \left[ \text{KL} \left( \pi_\theta(\cdot | s) || \pi_{\theta'}(\cdot | s) \right) \right] \leq \delta
\]
$$

> 这里的**不等式约束** 定义了策略空间中的一个 **"KL 球"**，被称为 **信任区域**。在这个区域中，可以认为当前学习策略和环境交互的状态分布与上一轮策略最后采样的状态 **分布一致**，进而可以使当前学习策略稳定提升。


### 14.1.3 近似求解
> 直接求解上式**带约束的优化问题**比较麻烦，**TRPO** 在其具体实现中做了一步近似操作来快速求解：

> 用$\theta_{k}$代替之前的$\theta$，表示这是第k次迭代之后的策略，然后在 **$\theta_{k}$处** 对上述的 **目标函数、约束** 进行泰勒展开:
- 一阶（**梯度Gradient**，给出斜率信息，指导更新的方向）：$\mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}\mathbb{E}_{a\sim\pi_{\theta_k}(\cdot|s)}\left[\frac{\pi_{\theta^{\prime}}(a|s)}{\pi_{\theta_k}(a|s)}A^{\pi_{\theta_k}}(s,a)\right]\approx grad^T(\theta^{\prime}-\theta_k)$
- 二阶（**海森矩阵Hessian Matrix**，给出曲率信息，指导更新的稳定）：$\mathbb{E}_{s\sim\nu^{\pi_{\theta_k}}}[D_{KL}(\pi_{\theta_k}(\cdot|s),\pi_{\theta^{\prime}}(\cdot|s))]\approx\frac{1}{2}(\theta^{\prime}-\theta_k)^TH(\theta^{\prime}-\theta_k)$

> 于是 **优化目标** 变为:
$$\theta_{k+1}=\arg\max_{\theta^{\prime}}g^T(\theta^{\prime}-\theta_k)$$
$$\frac{1}{2}(\theta^{\prime}-\theta_k)^TH(\theta^{\prime}-\theta_k)\leq\delta$$

> 再根据 **（Karush-Kuhn-Tucker，KKT）条件**:提供了一种求解最优解的有效途径。可直接导出上述问题的解：
$$\theta_{k+1}=\theta_k+\sqrt{\frac{2\delta}{g^TH^{-1}g}}H^{-1}g$$
- 其中$H^{-1}g$为 **自然梯度**，最初来源于Shun-Ichiro Amari在 1998 年的论文[Natural Gradient Works Efficiently in Learning](https://ieeexplore.ieee.org/document/6790500)中提出的 **自然梯度法（Natural Gradient Method）**
> 设$x=H^{-1}g$，根据矩阵性质可得 **参数更新方式**：
$$\theta_{k+1}=\theta_k+\sqrt{\frac{2\delta}{x^THx}}x$$

##### 另一种角度：
> 为了更快到达参数最优，假设满足 KL 距离约束的参数更新时的最大步长为$\beta$，则有：
$$\frac{1}{2}(\beta x)^{T}H(\beta x)=\delta$$
> 求解
$$\beta=\sqrt{\frac{2\delta}{x^{T}Hx}}$$
> 则 **参数更新方式** 亦为：
$$\theta_{k+1}=\theta_k+\sqrt{\frac{2\delta}{x^THx}}x$$

#### 共轭梯度法（Conjugate Gradient Method, CG）：用于高效求解对称正定矩阵 A 下的线性方程组：$Ax=b$
- 之所以叫做 **共轭梯度法**，是因为其梯度方向更新和普通梯度下降不同，它保留了“在函数空间中与之前梯度下降方向共轭”的性质，能避免重复探索，每一步都不会干扰之前已经优化过的方向（即：每步都在“新的维度”上优化）从而加速收敛。
> **KL 散度** 在旧策略处达到最小值，且关于新策略参数是局部严格凸的，因此它的 Hessian 是 **正定矩阵**。这个正定性保证了 **共轭梯度法** 可用，方向唯一、收敛稳定。
> **TRPO** 中神经网络表示的策略函数的参数数量都是成千上万的，计算和存储海森矩阵H的逆矩阵会 **耗费大量的内存资源和时间**，通过 **共轭梯度法**直接计算：$x=H^{-1}g$，可以解决资源消耗问题。


##### 共轭梯度法流程（求解 $\(Hx = g\)$，$\(H\)$ 对称正定）:
> **初始化**  
   - $\(x_0\)$：初始解（一般取零向量）  
   - $r_0 = g - H x_0$：初始残差（梯度）  
   - $p_0 = r_0$：初始搜索方向

> **迭代更新**（$k=0,1,2,\ldots$）

>计算步长：  
$$\alpha_k = \frac{r_k^T r_k}{p_k^T H p_k}$$  
>更新解：  
$$x_{k+1} = x_k + \alpha_k p_k$$  
>更新残差：  
$$r_{k+1} = r_k - \alpha_k H p_k$$  
>判断收敛：若 $\|r_{k+1}\|$ 足够小，停止迭代  
>计算比例因子：  
$$\beta_k = \frac{r_{k+1}^T r_{k+1}}{r_k^T r_k}$$  
>更新搜索方向：  
$$p_{k+1} = r_{k+1} + \beta_k p_k$$

> 其中，为了避免**显式构造 Hessian 矩阵**，节省内存和计算：
$$Hp=\nabla_\theta\left(\left(\nabla_\theta(D_{KL}^{\nu_k}(\pi_{\theta_k},\pi_{\theta^{\prime}}))\right)^T\right)p=\nabla_\theta\left(\left(\nabla_\theta(D_{KL}^{\nu_{k}}(\pi_{\theta_k},\pi_{\theta^{\prime}}))\right)^Tp\right)$$
> 即先用 **梯度和向量p点乘** 后再计算梯度

> 此时参数优化公式进化为:
$$\theta_{k+1}=\theta_k+\sqrt{\frac{2\delta}{x^THx}}x$$

#### 线性搜索（Line Search）：每一步的策略更新